In [2]:
using Symbolics
using LinearAlgebra
using Latexify
using Plots

In [8]:
D = 4 #dimensions
ζ_v = Vector{Float64}(undef,D)
ζ_t = Array{Float64}(undef,D,D)
g = Diagonal([-1.0, 1, 1, 1])

4×4 Diagonal{Float64, Vector{Float64}}:
 -1.0   ⋅    ⋅    ⋅ 
   ⋅   1.0   ⋅    ⋅ 
   ⋅    ⋅   1.0   ⋅ 
   ⋅    ⋅    ⋅   1.0

In [10]:
ζ_t = [-1 2 3 4;
        0 1 5 6;
        0 0 1 8;
        0 0 0 -1]
ζ_s = UpperTriangular(ζ_t)
ζ_s
println(tr(ζ_s'*g'*g*ζ_s))
ζ_h = Hermitian(ζ_s)
println(tr(ζ_h'*g'*g*ζ_h))
ζ_v'*g*ζ_v

158.0
312.0


0.0

In [39]:
function Φ(ζ_v, ζ_s, p)
    χ₀,χ₁,χ₂ = p
    ζ_h = Hermitian(ζ_s)
    μ = ζ_v'*g*ζ_v
    ν = (g*ζ_v)'*ζ_s*(g*ζ_v)
    τ = tr(ζ_s*g'*ζ_s*g)
    return χ₀*ζ_v'*g*ζ_v + χ₁*ν + χ₂*ν*τ
end

Φ (generic function with 1 method)

In [46]:
ζ_v = [-1,0.1,0,0]
ζ_t = [0 1 0 0;
       0 1 0 0;
       0 0 -1 0;
       0 0 0 0]
ζ_s = UpperTriangular(ζ_t)
ζ_h = Hermitian(ζ_s)

4×4 Hermitian{Int64, UpperTriangular{Int64, Matrix{Int64}}}:
 0  1   0  0
 1  1   0  0
 0  0  -1  0
 0  0   0  0

In [45]:
Φ(ζ_v, ζ_s, [1.,0.,0.])

-0.99

In [50]:
@variables zv[1:D], zs[1:D*(D+1)÷2], pv[1:3]

3-element Vector{Vector{Num}}:
 [zv₁, zv₂, zv₃, zv₄]
 [zs₁, zs₂, zs₃, zs₄, zs₅, zs₆, zs₇, zs₈, zs₉, zs₁₀]
 [pv₁, pv₂, pv₃]

In [11]:
JΦ_v = Symbolics.jacobian(Φ(zv, zs, pv),zv);
JΦ_v_exp = Symbolics.build_function(JΦ_v,,zv, zs,pv);
JΦ_v = eval(JΦ_v_exp[1]);

LoadError: UndefVarError: Φ not defined

In [72]:
function vector2symmat!(v,sm)
    M = length(sm[1,:]) # N = M*(M+1)/2 
    N = length(v)
    if M*(M+1)÷2 != N
        error("Dimension missmatch, M = $M, N= $N")
    end
    L = 1
    for i in 1:M
        for j in i:M
            sm[i,j] = v[L]
            sm[j,i] = sm[i,j]
            L=L+1
        end
    end
end

v = [1;2;3;4;5;6]
sm = zeros(3,3)

vector2symmat!(v,sm)

sm

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 2.0  4.0  5.0
 3.0  5.0  6.0